In [82]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
%matplotlib inline

In [83]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.utils.np_utils import to_categorical

In [84]:
dataset = 'mag-lev-images'
size = 64

In [85]:
def class_label(img):
    if '100-lido' in img:
        lbl = 0
    elif '100-caff' in img:
        lbl = 1
    elif '50-50-lido-caff' in img:
        lbl = 2
    return lbl

In [86]:
def images_with_label():
    images = []
    for i in tqdm(os.listdir(dataset)):
        path = os.path.join(dataset, i)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (size, size))
            images.append([np.array(img), class_label(i)])
    return images

In [115]:
images = images_with_label()

X = np.array([i[0] for i in images]).reshape(-1, size, size, 1)
y = np.array([i[1] for i in images])

100%|██████████| 29/29 [00:00<00:00, 82.36it/s]


In [116]:
kfold = StratifiedKFold(n_splits=7, shuffle=True)
cvscores = []

In [117]:
for train, test in kfold.split(X, y):    
    model = Sequential()

    model.add(InputLayer(input_shape=[size,size,1]))
    model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=5,padding='same'))

    model.add(Conv2D(filters=50,kernel_size=5,strides=1,padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=5,padding='same'))

    model.add(Conv2D(filters=80,kernel_size=5,strides=1,padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=5,padding='same'))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(3,activation='softmax'))
    optimizer = Adam(lr=1e-2)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(x=X[train],y=to_categorical(y[train]),epochs=50,batch_size=10)
    model.summary()
    
    # evaluate the model
    scores = model.evaluate(X[test], to_categorical(y[test]), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    print("\n\n")

Epoch 1/50
24/24 [==============================] - 1s 52ms/step - loss: 403.2200 - accuracy: 0.2500
Epoch 2/50
24/24 [==============================] - 0s 4ms/step - loss: 5.9195 - accuracy: 0.1667
Epoch 3/50
24/24 [==============================] - 0s 4ms/step - loss: 1.6819 - accuracy: 0.2917
Epoch 4/50
24/24 [==============================] - 0s 4ms/step - loss: 1.3837 - accuracy: 0.3333
Epoch 5/50
24/24 [==============================] - 0s 4ms/step - loss: 1.1067 - accuracy: 0.3333
Epoch 6/50
24/24 [==============================] - 0s 4ms/step - loss: 1.1091 - accuracy: 0.2917
Epoch 7/50
24/24 [==============================] - 0s 4ms/step - loss: 1.1067 - accuracy: 0.3333
Epoch 8/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0907 - accuracy: 0.4583
Epoch 9/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0919 - accuracy: 0.3750
Epoch 10/50
24/24 [==============================] - 0s 4ms/step - loss: 1.1073 - accuracy: 0.3750
Epoch 11/50
24/2

24/24 [==============================] - 0s 4ms/step - loss: 0.6766 - accuracy: 0.5417
Epoch 16/50
24/24 [==============================] - 0s 4ms/step - loss: 0.5833 - accuracy: 0.6667
Epoch 17/50
24/24 [==============================] - 0s 4ms/step - loss: 0.6954 - accuracy: 0.5417
Epoch 18/50
24/24 [==============================] - 0s 4ms/step - loss: 0.6878 - accuracy: 0.5000
Epoch 19/50
24/24 [==============================] - 0s 4ms/step - loss: 0.6414 - accuracy: 0.5833
Epoch 20/50
24/24 [==============================] - 0s 4ms/step - loss: 0.6739 - accuracy: 0.7083
Epoch 21/50
24/24 [==============================] - 0s 4ms/step - loss: 0.6467 - accuracy: 0.6667
Epoch 22/50
24/24 [==============================] - 0s 4ms/step - loss: 0.5726 - accuracy: 0.5417
Epoch 23/50
24/24 [==============================] - 0s 4ms/step - loss: 0.6008 - accuracy: 0.6250
Epoch 24/50
24/24 [==============================] - 0s 4ms/step - loss: 0.6668 - accuracy: 0.5833
Epoch 25/50
24/24 [===

24/24 [==============================] - 0s 4ms/step - loss: 1.0533 - accuracy: 0.4167
Epoch 30/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0079 - accuracy: 0.3750
Epoch 31/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0483 - accuracy: 0.2500
Epoch 32/50
24/24 [==============================] - 0s 4ms/step - loss: 0.9416 - accuracy: 0.6667
Epoch 33/50
24/24 [==============================] - 0s 4ms/step - loss: 0.8896 - accuracy: 0.6250
Epoch 34/50
24/24 [==============================] - 0s 4ms/step - loss: 0.9123 - accuracy: 0.5417
Epoch 35/50
24/24 [==============================] - 0s 4ms/step - loss: 0.8988 - accuracy: 0.5833
Epoch 36/50
24/24 [==============================] - 0s 4ms/step - loss: 0.8610 - accuracy: 0.5833
Epoch 37/50
24/24 [==============================] - 0s 4ms/step - loss: 0.9907 - accuracy: 0.5417
Epoch 38/50
24/24 [==============================] - 0s 4ms/step - loss: 1.2722 - accuracy: 0.2917
Epoch 39/50
24/24 [===

24/24 [==============================] - 0s 4ms/step - loss: 1.0898 - accuracy: 0.3333
Epoch 44/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0922 - accuracy: 0.3750
Epoch 45/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0837 - accuracy: 0.3333
Epoch 46/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0944 - accuracy: 0.2917
Epoch 47/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0879 - accuracy: 0.3333
Epoch 48/50
24/24 [==============================] - 0s 4ms/step - loss: 1.1019 - accuracy: 0.2500
Epoch 49/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0833 - accuracy: 0.4167
Epoch 50/50
24/24 [==============================] - 0s 4ms/step - loss: 1.1022 - accuracy: 0.3750
Model: "sequential_92"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_274 (Conv2D)          (None, 64, 64, 32)        832       

accuracy: 50.00%



Epoch 1/50
24/24 [==============================] - 1s 49ms/step - loss: 132.2765 - accuracy: 0.2917
Epoch 2/50
24/24 [==============================] - 0s 3ms/step - loss: 21.6847 - accuracy: 0.4167
Epoch 3/50
24/24 [==============================] - 0s 4ms/step - loss: 4.0419 - accuracy: 0.4167
Epoch 4/50
24/24 [==============================] - 0s 4ms/step - loss: 1.1391 - accuracy: 0.4167
Epoch 5/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0601 - accuracy: 0.5417
Epoch 6/50
24/24 [==============================] - 0s 4ms/step - loss: 1.3072 - accuracy: 0.3333
Epoch 7/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0450 - accuracy: 0.4167
Epoch 8/50
24/24 [==============================] - 0s 4ms/step - loss: 0.9134 - accuracy: 0.7083
Epoch 9/50
24/24 [==============================] - 0s 4ms/step - loss: 0.8913 - accuracy: 0.6250
Epoch 10/50
24/24 [==============================] - 0s 4ms/step - loss: 0.7749 - accuracy: 0.

24/24 [==============================] - 0s 4ms/step - loss: 1.0781 - accuracy: 0.3333
Epoch 16/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0867 - accuracy: 0.3333
Epoch 17/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0916 - accuracy: 0.4167
Epoch 18/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0745 - accuracy: 0.6250
Epoch 19/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0831 - accuracy: 0.4167
Epoch 20/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0876 - accuracy: 0.3333
Epoch 21/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0784 - accuracy: 0.5000
Epoch 22/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0751 - accuracy: 0.3750
Epoch 23/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0814 - accuracy: 0.4167
Epoch 24/50
24/24 [==============================] - 0s 4ms/step - loss: 1.0868 - accuracy: 0.3750
Epoch 25/50
24/24 [===

In [118]:
print("Average Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Average Accuracy: 46.43% (+/- 20.82%)
